# Data Preparation 

In [3]:
#Import packages
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import folium
#from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import anderson
from scipy.stats import norm
from matplotlib import pylab

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#import dataset created in Data Understanding
accidents = pd.read_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Accidents.csv",low_memory=False, encoding='utf-8')

In [3]:
accidents.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Region
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,3,17:42,12,E09000020,3,3218,6,30,0,-1,-1,0,0,1,1,2,2,0,0,1,1,E01002849,London
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,4,17:36,12,E09000020,4,450,3,30,6,2,5,0,0,5,4,1,1,0,0,1,1,E01002909,London
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,5,00:15,12,E09000020,5,0,6,30,0,-1,-1,0,0,0,4,1,1,0,0,1,1,E01002857,London
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,6,10:35,12,E09000020,3,3220,6,30,0,-1,-1,0,0,0,1,1,1,0,0,1,1,E01002840,London
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,2,21:13,12,E09000020,6,0,6,30,0,-1,-1,0,0,0,7,1,2,0,0,1,1,E01002863,London


## Select features clean and construct data

In [4]:
accidents.drop(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Accident_Index', 'LSOA_of_Accident_Location', 'Police_Force', 'Local_Authority_(District)',
                                   'Local_Authority_(Highway)', 'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number', 
                                   'Did_Police_Officer_Attend_Scene_of_Accident', '1st_Road_Number',
                'Pedestrian_Crossing-Physical_Facilities'], axis=1, inplace=True)

In [5]:
#date
#accidents['Date_time'] =  accidents['Date'] +' '+ accidents['Time']
#accidents['Date_time'] = pd.to_datetime(accidents["Date_time"])
accidents.drop(['Date','Time'],axis =1 , inplace=True)

#coordinates
#accidents["LatLon"] = list(zip(accidents["Latitude"], accidents["Longitude"]))
#accidents.drop(['Latitude','Longitude'],axis=1 , inplace=True)

#fill missing region values with "mode" and binary encoding of region
accidents["Region"] = accidents["Region"].fillna(accidents["Region"].mode()[0])
accidents = pd.get_dummies(accidents, columns=['Region'], drop_first=True)

#drop missing values (151)
accidents.dropna(inplace=True)

#Normalisation
scaler = MinMaxScaler()
accidents[["Latitude", "Longitude", "Number_of_Vehicles","Number_of_Casualties","Day_of_Week","1st_Road_Class","Road_Type",
           "Speed_limit","Junction_Control","Pedestrian_Crossing-Human_Control","Light_Conditions",
           "Weather_Conditions","Road_Surface_Conditions","Special_Conditions_at_Site","Carriageway_Hazards",
           "Urban_or_Rural_Area"]] = scaler.fit_transform(accidents[["Latitude", "Longitude", "Number_of_Vehicles","Number_of_Casualties","Day_of_Week","1st_Road_Class","Road_Type","Speed_limit","Junction_Control","Pedestrian_Crossing-Human_Control","Light_Conditions","Weather_Conditions","Road_Surface_Conditions","Special_Conditions_at_Site","Carriageway_Hazards","Urban_or_Rural_Area"]])


In [6]:
accidents.head()

,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Control,Pedestrian_Crossing-Human_Control,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Region_East of England,Region_London,Region_North East (England),Region_North West (England),Region_Scotland,Region_South East (England),Region_South West (England),Region_Wales,Region_West Midlands (England),Region_Yorkshire and The Humber
0,0.789488,0.145340,2,0.000000,0.0,0.333333,0.4,0.625,0.428571,0.0,0.333333,0.0,0.3,0.500000,0.125,0.125,0.0,0,1,0,0,0,0,0,0,0,0
1,0.787274,0.148197,3,0.000000,0.0,0.500000,0.6,0.250,0.428571,0.6,0.333333,0.5,0.2,0.333333,0.125,0.125,0.0,0,1,0,0,0,0,0,0,0,0
2,0.787840,0.148679,3,0.015152,0.0,0.666667,0.8,0.625,0.428571,0.0,0.333333,0.5,0.2,0.333333,0.125,0.125,0.0,0,1,0,0,0,0,0,0,0,0
3,0.791353,0.144726,3,0.000000,0.0,0.833333,0.4,0.625,0.428571,0.0,0.333333,0.0,0.2,0.333333,0.125,0.125,0.0,0,1,0,0,0,0,0,0,0,0
4,0.793212,0.145954,3,0.000000,0.0,0.166667,1.0,0.625,0.428571,0.0,0.333333,1.0,0.2,0.500000,0.125,0.125,0.0,0,1,0,0,0,0,0,0,0,0


In [7]:
accidents.shape

(1780515, 27)

In [8]:
accidents.dtypes

Longitude                            float64
Latitude                             float64
Accident_Severity                      int64
Number_of_Vehicles                   float64
Number_of_Casualties                 float64
Day_of_Week                          float64
1st_Road_Class                       float64
Road_Type                            float64
Speed_limit                          float64
Junction_Control                     float64
Pedestrian_Crossing-Human_Control    float64
Light_Conditions                     float64
Weather_Conditions                   float64
Road_Surface_Conditions              float64
Special_Conditions_at_Site           float64
Carriageway_Hazards                  float64
Urban_or_Rural_Area                  float64
Region_East of England                 uint8
Region_London                          uint8
Region_North East (England)            uint8
Region_North West (England)            uint8
Region_Scotland                        uint8
Region_Sou

In [9]:
#final dataset for Modeling
accidents.to_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Cleaned_dataset_accidents.csv", index=False, encoding='utf-8')

# Modeling

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier

In [4]:
accidents = pd.read_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Cleaned_dataset_accidents.csv",low_memory=False, encoding='utf-8')

In [10]:
#define influencing and response variable
X = accidents.drop("Accident_Severity", axis=1)
y = accidents["Accident_Severity"]
# Split the data into a training and test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

## Decision Tree 

In [14]:
dt = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 30, splitter='best')
dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
acc_decision = round(dt.score(X_test, y_test) * 100, 2)
sk_report = classification_report(digits = 6, y_true = y_test, y_pred = y_pred)

print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("Accuracy", acc_decision)
print("\n")
print(sk_report)

### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)



Cohen Kappa: 0.05699113873538275
Accuracy 80.05


              precision    recall  f1-score   support

           1   0.045830  0.014665  0.022220      6819
           2   0.198808  0.121252  0.150634     72642
           3   0.858589  0.920854  0.888632    454694

    accuracy                       0.800545    534155
   macro avg   0.367742  0.352257  0.353828    534155
weighted avg   0.758487  0.800545  0.777208    534155



Predicted,1,2,3,All
Actual,,,,
1,100,1136,5583,6819
2,455,8808,63379,72642
3,1627,34360,418707,454694
All,2182,44304,487669,534155


# Logistic Regression

In [15]:
clf = LogisticRegression().fit(X_train, y_train)
y_pred = clf.predict(X_test)
sk_report = classification_report(digits = 6, y_true = y_test, y_pred = y_pred)

print("Accuracy", round(accuracy_score(y_test, y_pred) * 100,2))
print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("\n")
print(sk_report)

### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 85.12
Cohen Kappa: 0.0008597824097933415


              precision    recall  f1-score   support

           1   0.000000  0.000000  0.000000      6819
           2   0.422680  0.000564  0.001127     72642
           3   0.851316  0.999908  0.919648    454694

    accuracy                       0.851238    534155
   macro avg   0.424665  0.333491  0.306925    534155
weighted avg   0.782156  0.851238  0.782994    534155



Predicted,2,3,All
Actual,,,
1,14,6805,6819
2,41,72601,72642
3,42,454652,454694
All,97,534058,534155


# KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train) 
y_pred = knn.predict(X_test)
sk_report = classification_report(digits = 6, y_true = y_test, y_pred = y_pred)

print("Accuracy", round(accuracy_score(y_test, y_pred) * 100,2))
print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("\n")
print(sk_report)
### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)


## Support Vector Machines

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
sk_report = classification_report(digits = 6, y_true = y_test, y_pred = y_pred)

print("Accuracy", round(accuracy_score(y_test, y_pred) * 100,2))
print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("\n")
print(sk_report)
### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)


# Naive Bayes

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
sk_report = classification_report(digits = 6, y_true = y_test, y_pred = y_pred)

print("Accuracy", round(accuracy_score(y_test, y_pred) * 100,2))
print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("\n")
print(sk_report)

### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

## Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
rf.score(X_test, y_test)
acc_rf = round(rf.score(X_test, y_test) * 100, 2)
sk_report = classification_report(digits=6, y_true=y_test, y_pred=y_pred)

print("Accuracy", acc_rf)
print("Cohen Kappa: "+str(cohen_kappa_score(y_test,y_pred)))
print("\n")
print(sk_report)

### Confusion Matrix 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)